# news_summary

## API key 로딩

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# 환경변수 로딩
load_dotenv(override=True)

# 메모리에 로딩된 값을 api_key 변수에 대입
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# print(api_key)

# OpenAI 객체 생성
client = OpenAI(api_key=OPENAI_API_KEY)

In [8]:
# 프롬프트
summary_prompt = """당신은 언어 이해 및 요약에 훈련된 고도로 숙련된 AI입니다.
다음 텍스트를 읽고 간결한 추상적 단락으로 요약했으면 합니다.
전체 텍스트를 읽을 필요 없이 토론의 요점을 이해하는 데 도움이 될 수 있는 일관되고
읽을 수 있는 요약을 제공하여 가장 중요한 요점을 유지하는 것을 목표로 합니다.
불필요한 세부 사항이나 접선 사항은 피하십시오."""
key_points_prompt = """당신은 정보를 핵심 포인트로 전달하는 데 특화된 능숙한 AI입니다.
다음 텍스트를 기반으로 논의되거나 언급된 주요 포인트를 확인하고 나열합니다.
이는 논의의 본질에 가장 중요한 아이디어, 결과 또는 주제가 되어야 합니다.
당신의 목표는 누군가가 읽을 수 있는 목록을 제공하여 이야기된 내용을 빠르게 이해하는 것입니다."""
action_items_prompt = """당신은 대화를 분석하고 행동 항목을 추출하는 데 있어 AI 전문가입니다.
본문을 검토하고 합의되거나 수행이 필요하다고 언급된 모든 작업, 과제 또는 행동을 식별하십시오.
이것들은 특정 개인에게 할당된 작업일 수도 있고 그룹이 취하기로 결정한 일반적인 행동일 수도 있습니다.
이러한 행동 항목을 명확하고 간결하게 나열하십시오."""
sentiment_prompt = """당신은 언어와 감정 분석에 전문성을 갖춘 AI로서 당신의 과제는 다음 텍스트의 감
정을 분석하는 것입니다.
토론의 전체적인 톤, 사용된 언어가 전달하는 감정, 단어와 구가 사용되는 맥락을 고려하십시오.
감정이 일반적으로 긍정적인지 부정적인지 중립적인지를 표시하고 가능한 한 당신의 분석에 대해 간략한 설명을 제
공하십시오."""


In [14]:
def text_extraction(text, prompt) :
    response = client.chat.completions.create(
        model='gpt-4-turbo',
        temperature=0,
        messages=[
            {"role" : "system", "content": prompt},
            {"role":"user", "content": text}
        ]
    )
    return response.choices[0].message.content

In [15]:
audio_file_path = "datas/news.mp3"
with open(audio_file_path, 'rb') as audio_file:
    transcription = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    response_format="text"
    )

In [11]:
transcription

'짧은 장마, 긴 폭염 탓에 말벌이 극성입니다. 119 대원들이 더 바빠졌습니다. 출동 현장에 김호영 기자도 동행했습니다. 건물 처마 밑에 커다란 벌집이 달려있고 수십 마리 말벌대가 우글댑니다. 보호복을 입은 소방대원이 스프레이를 뿌리자 벌들은 빠르게 흩어지고 미처 도망을 못 간 벌들은 바닥으로 떨어집니다. 공장 건물에 말벌대가 신고되면서 119 소방대원이 제거작업에 나선 겁니다. 불과 20미터 떨어진 곳. 이번에 신고된 벌집은 사무실 창문 바로 위에 달려 있습니다. 119 대원들은 사람들을 대피시킨 뒤 스프레이를 뿌려 벌집을 떼어냅니다. 벌들은 어두운 색을 공격하는 습성이 있기 때문에 119 대원들은 이렇게 하얀색의 말벌 보호복을 입고 벌이 들어가지 못하도록 빈틈없이 밴드를 붙입니다. 수요일에는 한 많게는 15건까지 출동하고 있습니다. 8에서 9월 달에 지금 왕성이 활동하고 그래서 쏘임사고도 벌 쏘임사고도 가장 많은... 지난달 소방에 벌집 제거 출동은 4만 4천 건으로 지난해보다 57%나 늘었습니다. 짧은 장마와 긴 폭염이 이어지며 벌들이 활동하기 좋은 환경이 조성된 겁니다. 소방당국은 말벌이 자주 나타날 땐 주변에 벌집이 있는지 잘 살펴 신고하고 벌이 공격해올 때는 현장에서 20미터 정도 거리까지 빨리 벗어나라고 조언했습니다. 채널A 뉴스 김호영입니다.\n'

In [16]:
abstract_summary = text_extraction(transcription, summary_prompt)
key_points = text_extraction(transcription, key_points_prompt)
action_items = text_extraction(transcription, action_items_prompt)
sentiment = text_extraction(transcription, sentiment_prompt)
news_data = {
    'abstract_summary': abstract_summary,
    'key_points': key_points,
    'action_items': action_items,
    'sentiment': sentiment
}
print(news_data)

{'abstract_summary': '짧은 장마와 긴 폭염으로 인해 말벌 활동이 증가하면서 소방대원들의 출동이 빈번해졌습니다. 말벌집 제거 요청은 지난해 대비 57% 증가한 44,000건에 달했습니다. 소방대원들은 특수 보호복을 착용하고 스프레이를 사용하여 벌집을 제거하는 작업을 수행하고 있으며, 말벌의 공격을 피하기 위해 주변을 잘 살피고 신속히 대피할 것을 권장하고 있습니다. 말벌은 특히 어두운 색을 공격하는 경향이 있어, 소방대원들은 하얀색 보호복을 착용합니다. 말벌의 활동이 가장 왕성한 8월과 9월에는 벌 쏘임 사고도 가장 많이 발생합니다.', 'key_points': '1. 짧은 장마와 긴 폭염으로 말벌 활동이 증가함.\n2. 소방대원들이 말벌 제거 작업에 더 자주 출동하고 있음.\n3. 말벌 제거 출동 건수가 지난해 대비 57% 증가하여 한 달에 44,000건에 달함.\n4. 소방대원들은 말벌 제거 시 특수 보호복을 착용하고 스프레이를 사용하여 벌집을 제거함.\n5. 소방당국은 말벌이 자주 나타나는 지역에서는 주변을 잘 살펴 벌집을 신고하고, 말벌 공격 시 빠르게 현장을 벗어날 것을 조언함.\n6. 말벌은 어두운 색을 공격하는 습성이 있어 소방대원들은 하얀색 보호복을 착용함.\n7. 8월과 9월에 말벌 활동이 가장 왕성하며 이 시기에 쏘임 사고도 가장 많이 발생함.', 'action_items': '1. 소방대원들은 말벌집 제거 작업을 계속 수행합니다.\n2. 시민들은 말벌이 자주 나타나는 지역에서 벌집이 있는지 주변을 잘 살펴보고 신고해야 합니다.\n3. 시민들은 말벌이 공격할 경우, 현장에서 20미터 이상 빨리 벗어나야 합니다.', 'sentiment': '이 텍스트의 전체적인 감정은 주로 중립적이며 경고적인 요소가 포함되어 있습니다. 텍스트는 말벌의 증가와 이로 인한 소방대원의 활동 증가를 설명하고 있으며, 이는 자연 현상과 그에 따른 사회적 대응을 보고하는 뉴스 보도의 특성을 반영합니다. 긍정적이거나 부정적인 감정보다는 정보 전달과 주의 권

In [22]:
type(news_data.values())




dict_values

In [ ]:
from docx import Document
doc = Document()

for key, value in news_data.items():
    # _로 나눈 단어들을 대문자로 만들고 공백으로 연결
    heading = ' '.join(word.capitalize() for word in key.split('_'))
    doc.add_heading(heading, level=1)
    doc.add_paragraph(value)
    # 섹션 사이에 줄바꿈 추가
    doc.add_paragraph()
    
# 워드 파일로 데이터 저장
doc.save('news_minutes.docx')